# lab12 - st121413

## In-lab exercises

1. Spend some time to understand the Transformer and Speech Recognition examples.
 - I did this in the main_\[modelname\].py and save the best model in /checkpoint for the next one
 - https://github.com/akraradets/2021JanRTML/tree/main/lab12
2. After training, test the two models on some data (text, audio) you provide yourself

## Transformer

In [1]:
from torchtext.data.utils import get_tokenizer
from myTransformer import *
from utils import *
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
tokenizer = get_tokenizer('basic_english')
vocab = load_vocab('checkpoint/transformer_vocab.pickle')
batch_size = 10
def data_process(raw_text_iter, vocab, tokenizer):
    data = [torch.tensor([vocab[token] for token in tokenizer(item)], dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

bptt = 35
text = "Visual Studio Code is a lightweight yet full featured cross platform IDE for software development that has recently caught up in terms of capabilities and popularity with other popular IDEs for Python such as PyCharm. It is reputed to be easier to configure and use, also. We'll give it a try this semester."
text_tensor = data_process(text, vocab, tokenizer)
text_batch = batchify(text_tensor, batch_size)

In [2]:
ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
model.load_state_dict(torch.load('checkpoint/transformer.pth'))
model.to(device)
criterion = nn.CrossEntropyLoss()
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

test_loss = evaluate(model, text_batch)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(test_loss, math.exp(test_loss)))

| End of training | test loss  8.54 | test ppl  5110.08


## Speech Recognition

## Homework

Since you're all extremely busy with your projects (RIGHT??), for the homework for this week,
write an explanation of the steps that would be needed to adapt the PyTorch transformer to
the speech recognition task. Be specific in how you would do the implementation, but you do
not need to ac